In [22]:
using JuMP, Gurobi, LinearAlgebra, Random, Statistics,MAT

In [6]:
J = 9
M = 2

TE = [18, 14, 12, 6, 36, 9, 9, 12, 12]
TC = [4, 4, 6, 6, 12, 6, 2, 2, 4]
CO = [3, 3, 4, 4, 8, 6, 2, 2, 4]
st = [2, 3, 4];
change = 5

5

In [7]:
v2 = Model(Gurobi.Optimizer)

@variable(v2, x[0:J,0:J,1:M], Bin)#ordem mesas
@variable(v2, y[1:J,1:M], Bin)#qual mesa
@variable(v2, z[0:J,0:J], Bin)#ordem corte
@variable(v2, cutt_change[0:J,0:J], Bin)#troca de mesa
@variable(v2, d1[1:J,1:J], Bin)#inter
@variable(v2, d2[1:J,1:J], Bin)#inter
@variable(v2, d[1:J,1:J], Bin)#inter
@variable(v2, C[0:J]≥0, Int)#tempo do corte
@variable(v2, E[0:J]≥0, Int)#tempo do enfesto
@variable(v2, time>=0)# tempo

@constraint(v2 , C[0]==0)#corte 0
@constraint(v2, E[0]==0)#enfesto 0
@constraint(v2 , [i=0:J,m=1:M], x[i,i,m]==0)#nenhuma ordem repete
@constraint(v2, [i=0:J], z[i,i]==0)#nenhum corte repete
@constraint(v2,[j=1:J], cutt_change[0,j]==0)

#Restrições de Ordem de Enfesto
@constraint(v2, [i=1:J], sum(y[i,m] for m in 1:M) == 1)#cada ordem passa 1x
@constraint(v2, [i=1:J,m=1:M], sum(x[i,j,m] for j in 0:J)==y[i,m])#toda ordem tem um sucessor
@constraint(v2, [j=1:J,m=1:M], sum(x[i,j,m] for i in 0:J)==y[j,m])#toda ordem tem um antecessor
@constraint(v2, [m=1:M], sum(x[0,j,m] for j in 1:J)≤ 1)#precisamos de uma primeira ordem

#Restriçoes de capacidade
@constraint(v2, [i=1:J,j=1:J,m=1:2], d1[i,j]≥(C[i]-E[j]+TE[j])/1000 - (2-y[i,m]-y[j,m])*1000)#checa se a ordem j começa antes da ordem i terminar
@constraint(v2, [i=1:J,j=1:J,m=1:2], d2[i,j]≥(C[j]-E[i]+TE[i])/1000 - (2-y[i,m]-y[j,m])*1000)#checa se a ordem j termina depois da ordem i comecar
@constraint(v2, [i=1:J,j=1:J], d[i,j]≥d1[i,j]+d2[i,j]-1.5)#checa se a ordem j começa antes da ordem i terminar e termina depois da ordem i começar temos intercessão
@constraint(v2, [i=1:J], sum(CO[j]*d[i,j] for j in 1:J)≤10)#restriçao de capacidade


# restrições de corte
@constraint(v2,[i=0:J,j=0:J], cutt_change[i,j]≤z[i,j])
@constraint(v2, [i=1:J], sum(z[i,j] for j in 0:J)==1)#cada corte tem um sucessor
@constraint(v2, [j=1:J], sum(z[i,j] for i in 0:J)==1)#cada corte tem um antecessor
@constraint(v2, fisrtcutt, sum(z[0,j] for j in 1:J)≤ 1)#precisamos de um primeiro corte
@constraint(v2, [i=0:J,j=0:J], sum(x[i,j,m] for m in 1:M) + sum(cutt_change[i,k] for k in 0:J) >= z[i,j])#restrição de balanço

#restrições de tempo
@constraint(v2, [i=0:J,j=1:J,m=1:2], E[j]≥E[i]+st[m]+TE[j]-(1-x[i,j,m])*10000)#enfesto deve terminar depois do ultimo junto ao tempo de setup
@constraint(v2, [i=0:J,j=1:J], C[j]≥C[i]+st[3]+change*cutt_change[i,j]+TC[j]-(1-z[i,j])*10000)#corte deve terminar dps do ultimo
@constraint(v2, [i=1:J], C[i]≥E[i]+st[3]+TC[i])



@constraint(v2, [i=1:J], time ≥ C[i]);

@constraint(v2, Bound, time ≥ 90)

@objective(v2, Min, time);

set_optimizer_attribute(v2, "LogFile", "Logv2_3")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 941647
Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br
Set parameter LogFile to value "Logv2_3"


In [8]:
optimize!(v2)

Set parameter LogFile to value "Logv2_3"
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br
Optimize a model with 1010 rows, 682 columns and 4953 nonzeros
Model fingerprint: 0x19ca35a5
Variable types: 1 continuous, 681 integer (661 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 134 rows and 96 columns
Presolve time: 0.01s
Presolved: 876 rows, 586 columns, 4185 nonzeros
Variable types: 0 continuous, 586 integer (567 binary)

Root relaxation: objective 9.000000e+01, 238 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

In [9]:
objective_value(v2)

106.0

In [13]:
value_x = zeros((10,10,2))
value_x[:,:,:] = value.(v2[:x])[:,:,:]

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 0:9
    Dimension 2, 0:9
    Dimension 3, Base.OneTo(2)
And data, a 10×10×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0

In [14]:
value_z = zeros((10,10))
value_z[:,:] = value.(v2[:z])[:,:]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 0:9
    Dimension 2, 0:9
And data, a 10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0

In [16]:
value_w= zeros((10,10))
value_w[:,:] = value.(v2[:cutt_change])[:,:]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 0:9
    Dimension 2, 0:9
And data, a 10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [17]:
value_d = zeros((9,9))
value_d[:,:] = value.(v2[:d])[:,:]

9×9 Matrix{Float64}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0

In [18]:
value_E = zeros(10)
value_E[:] = value.(v2[:E])[:]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:9
And data, a 10-element Vector{Float64}:
  0.0
 77.0
 57.0
 41.0
  8.0
 90.0
 19.0
 48.0
 30.0
 15.0

In [19]:
value_C = zeros(10)
value_C[:] = value.(v2[:C])[:]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:9
And data, a 10-element Vector{Float64}:
   0.0
  85.0
  77.0
  69.0
  18.0
 106.0
  29.0
  54.0
  48.0
  42.0

In [20]:
Dicionario = Dict(
"x" => value_x,
"z" => value_z,
"w" => value_w,
"d" => value_d,
"C" => value_C,
"E" => value_E        
)

Dict{String, Array{Float64}} with 6 entries:
  "w" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 …
  "C" => [0.0, 85.0, 77.0, 69.0, 18.0, 106.0, 29.0, 54.0, 48.0, 42.0]
  "x" => [0.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 …
  "z" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 …
  "E" => [0.0, 77.0, 57.0, 41.0, 8.0, 90.0, 19.0, 48.0, 30.0, 15.0]
  "d" => [1.0 1.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 1.0; 0.0 0.0 …

In [23]:
matwrite("v2.mat", Dicionario)